# Load Pickles Files

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import pickle

### Merge crop and trades DataFrame

In [3]:
def double_pivot(raw_df):
    '''
    Function that pivots the table twice to creating five new columns based on two columns.
    '''
    # pivot column 'element crops'
    df = raw_df.pivot_table(values=['value_crops'],\
                                  index=['area_code', 'area_crops', 'item_code_crops', 'item_crops', 'year', \
                                         'item_trades', 'element_trades', 'value_trades'],\
                                  columns=['element_crops']).reset_index()
    
    # deal with index to flatten
    df.columns = [' '.join(col).strip() for col in df.columns.values]

    # rename columns according to our nomenclature
    df.rename(columns= {'value_crops Area harvested': 'area_harvested', 'value_crops Production': 'production',\
                   'value_crops Yield':'yield'}, inplace=True)
    
    # pivot column 'element trades'
    df = df.pivot_table(values=['value_trades'],\
                        index=['area_code', 'area_crops', 'item_code_crops', 'item_crops', 'year', \
                               'item_trades', 'area_harvested', 'production', 'yield'],\
                        columns=['element_trades']).reset_index()
        
    # deal with index to flatten
    df.columns = [' '.join(col).strip() for col in df.columns.values]
    
    # rename columns according to our nomenclature
    df.rename(columns= {'value_trades Export Quantity': 'export_q', 'value_trades Export Value': 'export_v',\
                   'value_trades Import Quantity':'import_q', 'value_trades Import Value': 'import_v'}, inplace=True)
    
    return df

### Group crops

In [4]:
def load_group_crops_df():
    #Quickly load the data : 
    group_crops_df = pd.read_csv('data/GroupCrops.csv')
    #Column names to lower case
    group_crops_df.columns = map(lambda name : name.lower().replace(' ', '_'), group_crops_df.columns) 

    # Load the UN Comtrade Commodity Classifications :
    UNCCC_df = pd.read_excel('data/UNCCC.xlsx')
    # Keep the classification we are interested in (H4)
    HS12_df = UNCCC_df[UNCCC_df.Classification=='H4']
    
    # remove Nan values
    group_crops_cleaned_df = group_crops_df[~group_crops_df['hs12_code'].isnull()]
    
    # If the items is assigned mutliple labels, we only keep the first one
    HS12_Code = group_crops_cleaned_df['hs12_code'].str.split(', ',expand=True).loc[:,0]
    
    group_crops_cleaned_df = pd\
                            .concat([group_crops_cleaned_df,HS12_Code],axis=1)\
                            .drop(['factor','hs_code','hs07_code','cpc_code','hs12_code', \
                                   'item_group_code', 'item_group', 'item'],axis=1)
    
    group_crops_cleaned_df.rename(columns={0:'hs12_code'},inplace=True)
    
    # Add parent and child codes
    group_crops_cleaned_df['parent_group'] = group_crops_cleaned_df['hs12_code'].str[:2]#.astype(int)
    group_crops_cleaned_df['child_group'] = group_crops_cleaned_df['hs12_code'].str[:4]#.astype(int)
    
    # Add parent description
    group_crops_cleaned_df = group_crops_cleaned_df\
                        .merge(HS12_df[['Code', 'Description']], how='inner', left_on='parent_group', right_on='Code')\
                        .drop(['Code'], axis=1)

    group_crops_cleaned_df.rename(columns={'Description':'parent_description'}, inplace=True)
    
    # Add child description
    group_crops_cleaned_df = group_crops_cleaned_df\
                        .merge(HS12_df[['Code', 'Description']], how='inner', left_on='child_group', right_on='Code')\
                        .drop(['Code'], axis=1)

    group_crops_cleaned_df.rename(columns={'Description':'child_description'}, inplace=True)
    
    # drop duplicates
    group_crops_cleaned_df.drop_duplicates(inplace=True)
    
    return group_crops_cleaned_df

In [5]:
def final_df() :
    
    unpickled_df = pd.read_pickle("data/big_ass_df.pickle")
    
    # clean unpickled_df
    unpickled_df = unpickled_df.drop(['element_code_crops','flag','area_trades','element_code_trades'],axis=1)
    
    # reorganize columns by pivoting twice
    items_df = double_pivot(unpickled_df)
    
    # load group crops df
    groups_df = load_group_crops_df()
    
    # merge both 
    df_final = items_df.merge(groups_df, how='left', left_on='item_code_crops',right_on='item_code')\
                        .drop(['item_code_crops', 'item_code'], axis=1)
    
    return df_final

In [7]:
final_df = final_df()

In [8]:
# All items which don't have any child or parents matching (as we have done  a left join)
final_df[final_df.child_group.isnull()].item_trades.unique()

array(['Cigars, cheroots', 'Vegetables, preserved nes',
       'Vegetables in vinegar', 'Vegetables, dehydrated',
       'Vegetables, frozen', 'Vegetables, temporarily preserved',
       'Oil, vegetable origin nes',
       'Vegetables, homogenized preparations',
       'Vegetables, preserved, frozen', 'Waxes vegetable',
       'Feed, vegetable products nes',
       'Leeks, other alliaceous vegetables', 'Cereals, breakfast',
       'Flour, cereals', 'Juice, citrus, concentrated',
       'Juice, citrus, single strength', 'Flax fibre and tow',
       'Flax fibre raw', 'Oil, boiled etc', 'Oil, essential nes',
       'Oils, fats of animal nes', 'Oil, citronella',
       'Hair, goat, coarse', 'Pyrethrum, dried', 'Pyrethrum, extraction',
       'Peppermint'], dtype=object)

In [9]:
final_df[final_df.parent_group=='10'].child_description.unique()

array(['Wheat and meslin', 'Barley', 'Maize (corn)', 'Rye', 'Oats',
       'Rice', 'Buckwheat, millet and canary seeds; other cereals',
       'Grain sorghum'], dtype=object)

In [10]:
final_df.to_pickle('data/final_df.pkl')

### Country groups

In [11]:
# Load the dataset
df_country = pd.read_csv('Data/GroupsCountry.csv')
df_country.columns = map(lambda name : name.lower().replace(' ', '_'), df_country.columns) #Column names to lower case
df_country.head()

,country_group_code,country_group,country_code,country,m49_code,iso2_code,iso3_code
0,5100,Africa,4,Algeria,12,DZ,DZA
1,5100,Africa,7,Angola,24,AO,AGO
2,5100,Africa,53,Benin,204,BJ,BEN
3,5100,Africa,20,Botswana,72,BW,BWA
4,5100,Africa,233,Burkina Faso,854,BF,BFA


In [12]:
# Create a dictionnary that regroup all the countries in one country group
dicts = {}
list_group = df_country['country_group_code'].unique()
keys = list_group
for i in range(len(list_group)):
    dicts[list_group[i]]= df_country[df_country['country_group_code']==list_group[i]]['country_code'].unique()

In [13]:
# Create dictionnary for name + subregions
dicts_country = {}
keys = list_group
for i in range(len(list_group)):
    dicts_country[list_group[i]] = df_country['country_group'].unique()[i]

In [14]:
# Display all the suregions and their name
pd.DataFrame.from_dict(dicts_country,orient='index',columns=['regions']).head(15)

,regions
5100,Africa
5200,Americas
5300,Asia
5501,Australia and New Zealand
5206,Caribbean
5204,Central America
5301,Central Asia
5101,Eastern Africa
5302,Eastern Asia
5401,Eastern Europe


In [17]:
final_df

,area_code,area_crops,item_crops,year,item_trades,area_harvested,production,yield,export_q,export_v,import_q,import_v,hs12_code,parent_group,child_group,parent_description,child_description
0,1,Armenia,Wheat,1992,Wheat,65500.0,141483.0,21600.0,NaN,NaN,400000.0,60000.0,100111,10,1001,Cereals,Wheat and meslin
1,1,Armenia,Wheat,1993,Wheat,97900.0,217900.0,22257.0,NaN,NaN,408000.0,59000.0,100111,10,1001,Cereals,Wheat and meslin
2,1,Armenia,Wheat,1993,"Flour, wheat",97900.0,217900.0,22257.0,NaN,NaN,46000.0,9400.0,100111,10,1001,Cereals,Wheat and meslin
3,1,Armenia,Wheat,1994,Wheat,85697.0,152900.0,17842.0,NaN,NaN,327000.0,52000.0,100111,10,1001,Cereals,Wheat and meslin
4,1,Armenia,Wheat,1994,"Flour, wheat",85697.0,152900.0,17842.0,NaN,NaN,55000.0,14700.0,100111,10,1001,Cereals,Wheat and meslin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010908,5817,Net Food Importing Developing Countries,"Brazil nuts, with shell",2012,"Brazil nuts, shelled",11300.0,22600.0,20000.0,3789.0,23212.0,486.0,2712.0,080121,08,0801,"Fruit and nuts, edible; peel of citrus fruit o...","Nuts, edible; coconuts, Brazil nuts and cashew..."
1010909,5817,Net Food Importing Developing Countries,"Brazil nuts, with shell",2013,"Brazil nuts, shelled",11300.0,22800.0,20177.0,4347.0,29672.0,21.0,129.0,080121,08,0801,"Fruit and nuts, edible; peel of citrus fruit o...","Nuts, edible; coconuts, Brazil nuts and cashew..."
1010910,5817,Net Food Importing Developing Countries,"Brazil nuts, with shell",2014,"Brazil nuts, shelled",11300.0,22800.0,20177.0,6081.0,32877.0,131.0,371.0,080121,08,0801,"Fruit and nuts, edible; peel of citrus fruit o...","Nuts, edible; coconuts, Brazil nuts and cashew..."
1010911,5817,Net Food Importing Developing Countries,"Brazil nuts, with shell",2015,"Brazil nuts, shelled",11379.0,24647.0,21661.0,4940.0,35001.0,124.0,672.0,080121,08,0801,"Fruit and nuts, edible; peel of citrus fruit o...","Nuts, edible; coconuts, Brazil nuts and cashew..."


In [26]:
def badassfunction(df_import_export,group_country=None,country=None,group_crops=None,crops=None,year=None,by_crop=False,by_country=False):
    """
    From the original dataset, this function will processed the dataset 
    to return a summary of the desired crops, country and year
    INPUTS : 
    - df_import_export
    - group_country
    - country
    - group_crops
    - year
    OUTPUT : 
    - cleaned df
    """
    df_cleaned = df_import_export
    # Extract country
    if group_country != None :
        df_cleaned = df_cleaned[df_cleaned['area_code'].isin(dicts.get(group_country))]
    if country != None:
        df_cleaned = df_cleaned[df_cleaned['area_code']==country]
    if group_crops != None:
        df_cleaned = df_cleaned[df_cleaned['parent_group']==group_crops]
    if crops != None:
        df_cleaned = df_cleaned[df_cleaned['child_group']==crops]
    if year != None: 
        df_cleaned = df_cleaned[df_cleaned['year']==year]
     
    # Add new features
    df_cleaned['diff_q'] = df_cleaned['export_q'] - df_cleaned['import_q']
    df_cleaned['diff_v'] = df_cleaned['export_v'] - df_cleaned['import_v']

    # If feature is added above then add it name in that list
    list_features = ['export_q','export_v','import_q','import_v','diff_q','diff_v']
    #Group by item
    if by_crop:
        df_sum = df_cleaned.groupby(['item_crops','parent_group','child_group'])[list_features].sum().reset_index()

        df_max = df_cleaned.groupby(['item_crops','parent_group','child_group'])[list_features].max().reset_index()   
        df_max_index = df_cleaned.groupby(['item_crops','parent_group','child_group'])[list_features].idxmax().reset_index()

        df_min = df_cleaned.groupby(['item_crops','parent_group','child_group'])[list_features].min().reset_index()
        df_min_index =  df_cleaned.groupby(['item_crops','parent_group','child_group'])[list_features].idxmin().reset_index()
    
        merge_item = 'item_crops'
        replace_item = 'area_crops'

    if by_country:
        df_sum = df_cleaned.groupby(['area_crops','area_code'])[list_features].sum().reset_index()

        df_max = df_cleaned.groupby(['area_crops','area_code'])[list_features].max().reset_index()   
        df_max_index = df_cleaned.groupby(['area_crops','area_code'])[list_features].idxmax().reset_index()

        df_min = df_cleaned.groupby(['area_crops','area_code'])[list_features].min().reset_index()
        df_min_index =  df_cleaned.groupby(['area_crops','area_code'])[list_features].idxmin().reset_index()
    
        merge_item = 'area_crops'
        replace_item = 'item_crops'
    
    for i in range(len(list_features)):
        df_max_index.loc[:,list_features[i]] = df_import_export.loc[df_max_index.loc[:,list_features[i]],replace_item].values
    for i in range(len(list_features)):
        df_min_index.loc[:,list_features[i]] = df_import_export.loc[df_min_index.loc[:,list_features[i]],replace_item].values
    
    # merge max 
    df_max_merged = pd.merge(df_max.loc[:,[merge_item]+list_features],df_max_index.loc[:,[merge_item]+list_features],left_on=merge_item,right_on=merge_item,how='inner',suffixes=('_max','_max_names'))
    # merge min 
    df_min_merged = pd.merge(df_min.loc[:,[merge_item]+list_features],df_min_index.loc[:,[merge_item]+list_features],left_on=merge_item,right_on=merge_item,how='inner',suffixes=('_min','_min_names'))

    # merge total
    df_merged_tot = pd.merge(df_min_merged,df_max_merged, left_on=merge_item,right_on=merge_item)
    
    df_tot = pd.merge(df_sum, df_merged_tot, left_on=merge_item,right_on=merge_item)
    return df_tot

In [35]:
badassfunction(final_df,group_country=5100,group_crops=None,crops=None,year=2017,by_crop=True,by_country=False)

,item_crops,parent_group,child_group,export_q,export_v,import_q,import_v,diff_q,diff_v,export_q_min,...,import_q_max,import_v_max,diff_q_max,diff_v_max,export_q_max_names,export_v_max_names,import_q_max_names,import_v_max_names,diff_q_max_names,diff_v_max_names
0,"Almonds, with shell",08,0802,880.0,4073.0,10774.0,59188.0,-3260.0,-17384.0,0.0,...,6599.0,37684.0,-19.0,-100.0,Morocco,Morocco,Algeria,Algeria,Côte d'Ivoire,Côte d'Ivoire
1,"Anise, badian, fennel, coriander",09,0909,12650.0,28357.0,17526.0,46750.0,-4839.0,-18312.0,4.0,...,9730.0,32842.0,465.0,394.0,Egypt,Egypt,Egypt,Egypt,Ethiopia,Ethiopia
2,Apples,08,0808,616845.0,440858.0,250315.0,264853.0,374022.0,179332.0,0.0,...,189126.0,218363.0,552547.0,373275.0,South Africa,South Africa,Egypt,Egypt,South Africa,South Africa
3,Apricots,08,0809,9307.0,17012.0,6098.0,13357.0,7067.0,10453.0,0.0,...,2120.0,5946.0,4122.0,6716.0,South Africa,South Africa,Algeria,Algeria,South Africa,South Africa
4,Artichokes,07,0709,10677.0,13668.0,151.0,217.0,4390.0,4242.0,0.0,...,78.0,129.0,4386.0,4318.0,Egypt,Egypt,Kenya,Algeria,Tunisia,Tunisia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Vanilla,09,0905,2332.0,727061.0,196.0,1828.0,2136.0,725233.0,8.0,...,188.0,989.0,2139.0,701748.0,Madagascar,Madagascar,Uganda,Madagascar,Madagascar,Madagascar
101,"Vegetables, fresh nes",07,0706,169719.0,450303.0,153592.0,378458.0,134999.0,355955.0,0.0,...,75396.0,270000.0,55777.0,242984.0,Ethiopia,Ethiopia,Mauritania,Somalia,Ethiopia,Ethiopia
102,"Walnuts, with shell",08,0802,0.0,0.0,6565.0,23483.0,0.0,0.0,NaN,...,5833.0,19060.0,NaN,NaN,NaN,NaN,Morocco,Morocco,NaN,NaN
103,Watermelons,08,0807,199781.0,110567.0,3603.0,2027.0,158057.0,91451.0,0.0,...,1563.0,422.0,139039.0,80275.0,Morocco,Morocco,Mauritania,Mauritania,Morocco,Morocco
